# ETL_ CREDITS
TRANSFORMACIONES: 
- Eliminación de campos innecesarios en el análisis.<P>
- Cambio de nombres.<P>
- Cambio de tipos de datos.<P>
- Verificar los valores nulos del campo. <P>
- Eliminación de valores duplicados.<P>

In [5]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import ast

### 0.1. Carga de los datos

In [ ]:
credits_test  = pd.read_csv('datasets/credits.csv', sep=',',quotechar='"', encoding='utf-8' )

In [9]:
print(credits_test.shape)

(45476, 3)


Cambio de nombre columna 'id'=> 'IdMovie'

In [10]:
credits_test.rename(columns={'id':'IdMovie'}, inplace=True)

Se verifica

In [11]:
credits_test.head(1)

,cast,crew,IdMovie
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


### 0.2. Se obtiene dataframe CREW y dataframe CAST

In [12]:
# Se asignan a nuevos dataframes y se eliminan columnas no necesarias en cada uno
cast_df = credits_test.drop	(columns = "crew")
crew_df = credits_test.drop(columns = "cast")

In [13]:
type(cast_df)

pandas.core.frame.DataFrame

In [14]:
# se ordenan columnas, con Id como primera
cast_df = cast_df[['IdMovie', 'cast']]
crew_df= crew_df[['IdMovie', 'crew']]

In [15]:
# Se verifica
cast_df.head(1)

,IdMovie,cast
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)',..."


In [16]:
crew_df.head(1)

,IdMovie,crew
0,862,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


Funcion para desanidar

In [17]:
def desanidar_concat(df, col):
    # Convertir cadenas de texto en listas/diccionarios si es necesario
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Explode para listas de diccionarios o manejar diccionarios directamente
    if df[col].apply(lambda x: isinstance(x,list)).any(): 
        col_norm = pd.json_normalize(df[col].explode().dropna()).reset_index(drop=True)
    else: # para diccionarios
        col_norm = pd.json_normalize(df[col].dropna()).reset_index(drop=True)
    
    # Rename las nuevas columnas
    col_norm = col_norm.add_suffix(f'_{col}')

    #Elimina columna original, agrega columnas nuevas al dataframe
    #df = pd.concat([df.drop(columns=[col]) , col_norm], axis=1)
    new_df= df.drop(columns=col).join(col_norm)

    return new_df

In [18]:
cast_df= desanidar_concat(cast_df, "cast")
crew_df=desanidar_concat(crew_df, 'crew')

In [19]:
cast_df.head(1)

,IdMovie,cast_id_cast,character_cast,credit_id_cast,gender_cast,id_cast,name_cast,order_cast,profile_path_cast
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg


In [20]:
crew_df.head(1)

,IdMovie,credit_id_crew,department_crew,gender_crew,id_crew,job_crew,name_crew,profile_path_crew
0,862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg


## 1. TRANSFORMACION - cast_df

In [21]:
cast_df.head(2)

,IdMovie,cast_id_cast,character_cast,credit_id_cast,gender_cast,id_cast,name_cast,order_cast,profile_path_cast
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,8844,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg


### Eliminación de columnas

COLUMNAS:
- credit_id_cast, gender_cast, order_cast, profile_path_cast

In [22]:
columnas = [ 'credit_id_cast', 'gender_cast', 'order_cast', 'profile_path_cast']
cast_df = cast_df.drop(columnas, axis=1) 


In [23]:
cast_df.head(2)

,IdMovie,cast_id_cast,character_cast,id_cast,name_cast
0,862,14,Woody (voice),31,Tom Hanks
1,8844,15,Buzz Lightyear (voice),12898,Tim Allen


### Valores nulos

In [24]:
cast_df.isnull().sum()

IdMovie           0
cast_id_cast      0
character_cast    0
id_cast           0
name_cast         0
dtype: int64

In [25]:
cast_df.shape

(45476, 5)

### Cambiar nombre y tipo de dato

Cambio de nombre columnas

In [26]:
columnas = {'cast_id_cast':'cast_id', 'character_cast':'character', 
            'id_cast':'id_actor','name_cast':'name_actor'}

cast_df.rename(columns=columnas, inplace=True)

Tipo de datos

In [27]:
cast_df.dtypes

IdMovie        int64
cast_id        int64
character     object
id_actor       int64
name_actor    object
dtype: object

### Valores duplicados (identificadores únicos)

Cantidad de valores duplicados, luego de aparecer por segunda vez

In [28]:
duplicados= cast_df[cast_df["IdMovie"].duplicated()]
print(len(duplicados))
# duplicados

44


Eliminación de duplicados

In [29]:
# keep='first' se mantiene el id original, y se elimina el duplicado
cast_df.drop_duplicates(subset='IdMovie', keep='first', inplace=True)

Se verifica inexistencias de duplicados

In [30]:
duplicados= cast_df[cast_df["IdMovie"].duplicated()]
print(len(duplicados))
duplicados

0


,IdMovie,cast_id,character,id_actor,name_actor


## 2. TRANSFORMACION - crew_df

In [31]:
crew_df.head(2)

,IdMovie,credit_id_crew,department_crew,gender_crew,id_crew,job_crew,name_crew,profile_path_crew
0,862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,8844,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg


### Eliminación de columnas

COLUMNAS:
- credit_id_crew, gender_crew, profile_path_crew

In [32]:
columnas = [ 'credit_id_crew', 'gender_crew', 'profile_path_crew']
crew_df = crew_df.drop(columnas, axis=1) 

In [33]:
crew_df.head(1)

,IdMovie,department_crew,id_crew,job_crew,name_crew
0,862,Directing,7879,Director,John Lasseter


### Valores nulos

In [34]:
# Análisis 
crew_df.isnull().sum()

# No hay valores nulos

IdMovie            0
department_crew    0
id_crew            0
job_crew           0
name_crew          0
dtype: int64

### Tipo de datos

In [35]:
crew_df.dtypes

IdMovie             int64
department_crew    object
id_crew             int64
job_crew           object
name_crew          object
dtype: object

### Valores duplicados

In [36]:
duplicados= crew_df[crew_df["IdMovie"].duplicated()]
print(len(duplicados))

44


In [37]:
#Se eliminan valores duplicados
crew_df.drop_duplicates(subset='IdMovie', keep='first', inplace=True)

In [38]:
duplicados= crew_df[crew_df["IdMovie"].duplicated()]
print(len(duplicados))

0


## 3. EXPORTACIÓN DATASET FINAL 

In [140]:
crew_df.head(2)

,IdMovie,department_crew,id_crew,job_crew,name_crew
0,862,Directing,7879,Director,John Lasseter
1,8844,Writing,12891,Screenplay,Joss Whedon


In [141]:
cast_df.head(2)

,IdMovie,cast_id,character,id_actor,name_actor
0,862,14,Woody (voice),31,Tom Hanks
1,8844,15,Buzz Lightyear (voice),12898,Tim Allen


In [148]:
# Exportación
crew_df.to_parquet('datasets/credits_crew.parquet', engine='pyarrow')
cast_df.to_parquet('datasets/credits_cast.parquet', engine='pyarrow')

- Concatenación movie_cast
- Concatenación movie_crew <p>
Union mediante **'IdMovie'**

In [ ]:
movie_df = pd.read_parquet('datasets/movie_dataset_final.parquet', engine= 'pyarrow')
credits_cast_api = pd.read_parquet('datasets/credits_cast.parquet', engine= 'pyarrow')
credits_crew_api = pd.read_parquet('datasets/credits_crew.parquet', engine= 'pyarrow')

In [41]:
movie_cast = pd.merge(movie_df, credits_cast_api, on ='IdMovie')
movie_crew = pd.merge(movie_df, credits_crew_api, on ='IdMovie')


Exportamos

In [ ]:
movie_crew.to_parquet('datasets/movie_crew.parquet', engine='pyarrow')
movie_cast.to_parquet('datasets/movie_cast.parquet', engine='pyarrow')